#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [1]:
import pandas as pd
data_path = '../module_3/data/supermarket_sales.csv'
df = pd.read_csv(data_path)

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [25]:
total_profit_by_branch = df.groupby("Branch")["gross income"].sum()
branch =total_profit_by_branch.sort_values(ascending=False).index[0]
print(total_profit_by_branch,  '\n''\n' "The most profitable branch is", branch )

Branch
A    5057.1605
B    5057.0320
C    5265.1765
Name: gross income, dtype: float64 

The most profitable branch is C


2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [26]:
total_profit_by_branch_city = df.groupby(by=["Branch", "City"])["gross income"].sum()
city = total_profit_by_branch_city.sort_values(ascending=False).index[0][1]
print("The most profitable branch", branch, 'is loacated in', city )

The most profitable branch C is loacated in Naypyitaw


3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [29]:
invoices_by_category = pd.pivot_table(df,index='Branch' , columns='Product line', values='Quantity', aggfunc='count')
invoices_by_category

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [39]:
invoices_by_category['Electronic accessories'].sort_values(ascending=False).index[0]

'A'

4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


1. Customers with membership status tend to make more purchases compared to non-members.

In [79]:
membership_analysis  = pd.pivot_table(df,columns='Customer type' , index=['Branch'], values=['Quantity'], aggfunc=['sum'])
membership_analysis 

sum       
              Quantity       
Customer type   Member Normal
Branch                       
A                  964    895
B                  924    896
C                  897    934

In Branch A and Branch B, members made slightly more purchases than non-members. 
* So the hypothesis is correct and Branch C should promote memebership benefits. For example exclusive discounts, early access to sales or loylty rewards

2. Products with higher customer ratings are purchased more frequently than those with lower ratings.

In [80]:
def category_mapping(val):
    if val >= 8:
        return 'High'
    elif val >= 6:
        return 'Medium'
    else:
        return 'Low'
df["Rating Category"] = df.Rating.apply(category_mapping).astype('category')

In [101]:
rating_analysis  = pd.pivot_table(df,index=['Branch', 'Rating Category'], values=['Quantity', 'gross income'], aggfunc=['sum'])


In [95]:
rating_analysis.columns = ['Quantity', 'Income']

In [97]:
rating_analysis['ATP'] = rating_analysis['Income'] / rating_analysis['Quantity']
rating_analysis['ATP'] = rating_analysis['ATP'].round(2)
rating_analysis

Quantity     Income   ATP
Branch Rating Category                           
A      High                  609  1589.6435  2.61
       Low                   592  1615.5670  2.73
       Medium                658  1851.9500  2.81
B      High                  568  1514.6555  2.67
       Low                   622  1792.2685  2.88
       Medium                630  1750.1080  2.78
C      High                  661  1960.4430  2.97
       Low                   577  1700.7440  2.95
       Medium                593  1603.9895  2.70

* Branch C has the highest ATP (2.97) and the highest total income among the high rating categories.
* Branch B has a higher ATP compared to Branch A, suggesting that low-rated products are more expensive per transaction in Branch B.
* Branch A has the highest ATP (2.81) among medium rating products.

Suggestions:
* Branches should promote high-rated products
* Branch C has consistently high ATP across all rating categories, indicating that customers are willing to pay more. The business should analyze why this branch performs better and consider replicating successful strategies in other branches.

3. rating_analysis  = pd.pivot_table(df,index=['Branch', 'Rating Category'], values=['Quantity', 'gross income'], aggfunc=['sum'])

In [110]:
def price_mapping(val):
    if val >= 75:
        return 'High'
    elif val >= 45:
        return 'Medium'
    else:
        return 'Low'
df["Price Category"] = df['Unit price'].apply(price_mapping).astype('category')

In [111]:
price_analysis  = pd.pivot_table(df, columns='Price Category',index=['Branch'], values=['Quantity', 'Unit price'], aggfunc=['sum'])
price_analysis

sum                                         
               Quantity             Unit price                  
Price Category     High  Low Medium       High      Low   Medium
Branch                                                          
A                   455  721    683    7626.77  3551.72  7447.00
B                   539  702    579    8225.85  3473.12  6779.91
C                   599  663    569    9189.37  3315.59  6062.80

* Branch C shows the highest revenue from high-priced items. It should continue to promote those items
* Branch A has high sales volume but relatively lower revenue from low-priced items. It should adjust pricese for those items.
* Branch B and Branch C show significant revenue from medium-priced items. Adjusting the marketing focus or promotional strategies for medium-priced items could boost sales across branche